# SQL-Based Validation Test

## Notes

Made the following local database changes so we could test out the SQL validations:
* made a copy of the object_class table and put it in the staging database

The queries are running in branch __sql_validation_tests__.

SQL Gotchas:
* Postgres doesn't like double quotes for strings
* Be careful with NULL comparisons

## B11
Must be valid 3-digit object class as defined in OMB Circular A-11 Section 83.6, or a 4-digit code which includes a 1-digit prefix that distinguishes direct, reimbursable, and allocation obligations. Do not include decimal points when reporting in the Schema.

In [76]:
# setup
from dataactcore.models.stagingModels import Appropriation, ObjectClassProgramActivity
from dataactcore.models.stagingInterface import StagingInterface
from sqlalchemy import text

# run test data script that creates a fake submission with 3 TASs
%run fake_data.py
print ('Submission id for test data = {}'.format(submission_id))

# get interface to staging db
db = StagingInterface()

# since the sql rule table doesn't exist in this branch,
# create a dictionary to use as a stand-in

# Part 1 of B11: find rows with an invalid object class
rules = []
sql_rule = {}
sql_rule['rule_sql_id'] = 1
sql_rule['rule_sql'] = 'SELECT awdf.row, awdf.objectclass '\
'FROM award_financial '\
'AS awdf WHERE awdf.submission_id = {} '\
'AND RIGHT(awdf.objectclass, 3) NOT IN ('\
'SELECT object_class_code FROM object_class);'
sql_rule['rule_number'] = 'B11'
sql_rule['rule_description'] = 'Must be valid 3-digit object class as defined in OMB Circular A-11.'
sql_rule['rule_error_message'] = ''
sql_rule['rule_critical_flag'] = True
sql_rule['rule_crossfile_flag'] = False
sql_rule['file_id'] = 4
rules.append(sql_rule)


# Part 2 of B11: for 4 digit object classes check direct/reimbursable
sql_rule['rule_sql_id'] = 2
sql_rule['rule_sql'] = "SELECT awdf.row, awdf.objectclass FROM award_financial awdf WHERE awdf.submission_id = {} AND LENGTH(awdf.objectclass) = 4 AND ((RIGHT(awdf.objectclass,1) = '1' AND awdf.bydirectreimbursablefundingsource IS DISTINCT FROM 'D') OR (RIGHT(awdf.objectclass,1) = '2' AND awdf.bydirectreimbursablefundingsource IS DISTINCT FROM 'R'))"
sql_rule['rule_number'] = 'B11'
sql_rule['rule_description'] = 'Object class code which includes a 1-digit prefix must correctly distinguishes direct, reimbursable, and allocation obligations.'
sql_rule['rule_error_message'] = ''
sql_rule['rule_critical_flag'] = True
sql_rule['rule_crossfile_flag'] = False
sql_rule['file_id'] = 4
rules.append(sql_rule)

Submission id for test data = 5429


In [74]:
# loop through the sql rules "table"
for r in rules:
    sql = r['rule_sql']
    result = db.engine.execute(sql.format(submission_id))
    if result.returns_rows:
        cols = result.keys()
        cols.remove('row')
        print ('\nThere were {} errors for rule {}\n{}:\n'.format(
                result.rowcount, r['rule_number'], r['rule_description']))
        for row in result:
            row_number = row['row']
            values = ''
            for c in cols:
                values = values + '{}: {}, '.format(c, row[c])
                # test scenario where there is more than 1 problematic column
                values = values + '{}: {}'.format('fake col','bad data!')
            # Print statement to demo how we could construct an error message
            # using the current validator format by combining info in the 
            # sql rule table with data being returned by the validation query
            print ('Row {} failed validation {}. Columns with errors: {}'.format(
                    row_number, r['rule_number'], values))
    else:
        print ('Submission passed this rule, woo!')


There were 33 errors for rule B11
Object class code which includes a 1-digit prefix must correctly distinguishes direct, reimbursable, and allocation obligations.:

Row 7797 failed validation B11. Columns with errors: objectclass: 2251, fake col: bad data!
Row 139 failed validation B11. Columns with errors: objectclass: 1252, fake col: bad data!
Row 1461 failed validation B11. Columns with errors: objectclass: 2251, fake col: bad data!
Row 252 failed validation B11. Columns with errors: objectclass: 1252, fake col: bad data!
Row 2361 failed validation B11. Columns with errors: objectclass: 2251, fake col: bad data!
Row 6734 failed validation B11. Columns with errors: objectclass: 2251, fake col: bad data!
Row 306 failed validation B11. Columns with errors: objectclass: 1252, fake col: bad data!
Row 4705 failed validation B11. Columns with errors: objectclass: 2251, fake col: bad data!
Row 4593 failed validation B11. Columns with errors: objectclass: 1111, fake col: bad data!
Row 6040 


## B12
Reimbursable flag indicator is required when reporting obligation or outlay USSGL account balances (excluding downward adjustments USSGL accounts). Valid values are "R" for "ReimbursableFundingSource" and "D" for "DirectFundingSource".


### B9
Must be a valid program activity name for the corresponding TAS/TAFS as defined in Section 82 of OMB Circular A-11.